<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/stormglass_io(10request_per_day).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install openmeteo-requests requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.3/684.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.6 MB/s eta 0:00:00


In [14]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# --- CONFIG ---
API_KEY = '6e31b8d8-5614-11ef-aa85-0242ac130004-6e31b932-5614-11ef-aa85-0242ac130004'
CITY = {"name": "Turku", "lat": 60.45, "lng": 22.26}

def run_auto_fetch():
    all_data = []
    # Set your starting point here (change this tomorrow to where you left off)
    current_start = datetime(2024, 1, 1)
    end_of_year = datetime(2024, 12, 31)

    headers = {'Authorization': API_KEY}

    print(f"🚀 Starting automated download for {CITY['name']}...")

    while current_start < end_of_year:
        current_end = current_start + timedelta(days=5)

        params = {
            'lat': CITY['lat'], 'lng': CITY['lng'],
            'params': 'airTemperature,windSpeed',
            'start': int(current_start.timestamp()),
            'end': int(current_end.timestamp()),
            'source': 'sg'
        }

        response = requests.get('https://api.stormglass.io/v2/weather/point', params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()

            # Extract hours
            for hour in data.get('hours', []):
                all_data.append({
                    'Time': hour['time'],
                    'Temp_C': hour['airTemperature']['sg'],
                    'Wind_ms': hour['windSpeed']['sg']
                })

            # Check Quota from the meta tag
            meta = data.get('meta', {})
            used = meta.get('requestCount')
            total = meta.get('dailyQuota')
            print(f"✅ Extracted: {current_start.date()} to {current_end.date()} | Quota: {used}/{total}")

            if used >= total:
                print("🛑 Daily quota reached! Stopping for today.")
                break

            current_start = current_end + timedelta(hours=1)
            time.sleep(1) # Short pause between requests

        elif response.status_code == 429:
            print("🛑 Error 429: Quota exceeded according to server.")
            break
        else:
            print(f"❌ Failed at {current_start}: {response.status_code}")
            break

    # Save progress
    if all_data:
        df = pd.DataFrame(all_data)
        filename = f"{CITY['name'].lower()}_2024_progress.csv"
        df.to_csv(filename, index=False)
        print(f"💾 Progress saved to {filename} ({len(df)} hours total)")

run_auto_fetch()

🚀 Starting automated download for Turku...
✅ Extracted: 2024-01-01 to 2024-01-06 | Quota: 9/10
✅ Extracted: 2024-01-06 to 2024-01-11 | Quota: 10/10
🛑 Daily quota reached! Stopping for today.
💾 Progress saved to turku_2024_progress.csv (242 hours total)
